In [4]:
#pip install yt-dlp

In [ ]:
#자막 확인 코드 - 필요시만 확인 두번째 코드만 실행 시키면 됨.
import yt_dlp

def check_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            subtitles = info.get('subtitles', {})
            auto_subs = info.get('automatic_captions', {})
            
            print("\n[INFO] ▶️ 사용자 제공 자막:")
            for lang, tracks in subtitles.items():
                print(f"{lang}: {tracks}")
                
            print("\n[INFO] ⚙️ 자동 생성 자막:")
            for lang, tracks in auto_subs.items():
                print(f"{lang}: {tracks}")
                
            if not subtitles and not auto_subs:
                print("[WARNING] 자막이 없습니다!")
                
    except Exception as e:
        print(f"[ERROR] 자막 조회 실패: {e}")

# 테스트
video_url = "https://www.youtube.com/watch?v=rCcxXxoerjg"
check_subtitles(video_url)

In [24]:
import os
import yt_dlp
import shutil
import subprocess
import re

# 1️⃣ ffmpeg 경로 설정 (Windows 환경)
ffmpeg_path = os.environ.get("FFMPEG_PATH", r"C:\ffmpeg\bin")
if os.path.exists(ffmpeg_path):
    os.environ["PATH"] += os.pathsep + ffmpeg_path
else:
    print("[ERROR] ffmpeg 경로를 확인해 주세요!")
    exit(1)


# 2️⃣ 유튜브 동영상 형식 및 자막 코드 확인
def list_formats_and_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)

            print("\n🖼️ [INFO] ▶️ 사용 가능한 영상 형식:")
            for fmt in info['formats']:
                res = fmt.get('resolution', 'Unknown')
                size = fmt.get('filesize', 'Unknown')
                ext = fmt.get('ext', 'N/A')
                print(f"  🎞️ ID: {fmt['format_id']}, 해상도: {res}, 크기: {size} bytes, 형식: {ext}")

            subtitles = info.get('subtitles', {})
            auto_subs = info.get('automatic_captions', {})

            print("\n🔤 [INFO] ▶️ 사용자 제공 자막 언어:")
            for lang, tracks in subtitles.items():
                print(f"  🌐 {lang}: {tracks}")

            print("\n⚙️ [INFO] ▶️ 자동 생성 자막 언어:")
            for lang, tracks in auto_subs.items():
                print(f"  🤖 {lang}: {tracks}")

            if not subtitles and not auto_subs:
                print("⚠️ [WARNING] ▶️ 자막이 없는 영상입니다! (영상은 자막 없이 다운로드됨)")

            return bool(subtitles or auto_subs)

    except Exception as e:
        print(f"❌ [ERROR] ▶️ 형식 및 자막 코드 조회 실패: {e}")
        return False


# 3️⃣ 유튜브 동영상 다운로드 (한국어+영어 자막 포함)
def download_video_with_subtitles(video_url):
    try:
        with yt_dlp.YoutubeDL({'skip_download': True}) as ydl:
            info = ydl.extract_info(video_url, download=False)
            video_title = info.get('title', 'unknown_video')
            # 디렉터리 이름으로 부적합한 문자 제거 및 길이 제한 (최대 50자)
            safe_title = re.sub(r'[\\/:*?"<>|]', '_', video_title.replace(' ', '_'))[:50]

        # 영상별 폴더 생성
        video_folder = os.path.join("downloads", safe_title)
        os.makedirs(video_folder, exist_ok=True)

        # 자막 저장 디렉토리 미리 생성 (버그 방지)
        subtitle_folder = os.path.join(video_folder, "subtitles")
        os.makedirs(subtitle_folder, exist_ok=True)

        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best',
            'outtmpl': f'{video_folder}/{safe_title}.%(ext)s',
            'ffmpeg_location': ffmpeg_path,
            'writesubtitles': True,
            'writeautomaticsub': True,
            'subtitleslangs': ['en', 'ko'],
            'subtitlesformat': 'vtt',
            'embedsubs': True,
            'keepvideo': True,
            'postprocessors': [
                {'key': 'FFmpegVideoConvertor', 'preferedformat': 'mp4'},
                {'key': 'FFmpegEmbedSubtitle'}
            ],
            'noplaylist': True
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
            print(f"\n✅ [INFO] ▶️ 다운로드 완료: {safe_title} (한국어 & 영어 자막 포함)")
        return safe_title

    except Exception as e:
        print(f"❌ [ERROR] ▶️ 다운로드 실패: {video_url}, 오류: {e}")
        return None


# 4️⃣ 다운로드 후 자막 파일 백업
def backup_subtitles(video_title):
    src_folder = os.path.join("downloads", video_title)
    dest_folder = os.path.join(src_folder, "subtitles")
    os.makedirs(dest_folder, exist_ok=True)

    subtitle_files = [f"{video_title}.ko.vtt", f"{video_title}.en.vtt"]
    backed_up = False

    for file in subtitle_files:
        src_path = os.path.join(src_folder, file)
        dest_path = os.path.join(dest_folder, file)
        if os.path.exists(src_path):
            shutil.copy2(src_path, dest_path)
            print(f"🗂️ [INFO] ▶️ 자막 파일 별도 보관 완료: {dest_path}")
            backed_up = True

    if not backed_up:
        print("⚠️ [WARNING] ▶️ 자막 파일을 찾을 수 없습니다. 자막이 영상에만 포함되었을 수 있습니다.")


# 5️⃣ 다운로드된 자막 파일 확인
def print_subtitle_location(video_title):
    subtitle_folder = os.path.join("downloads", video_title, "subtitles")
    subtitle_files = [
        os.path.join(subtitle_folder, f"{video_title}.ko.vtt"),
        os.path.join(subtitle_folder, f"{video_title}.en.vtt")
    ]
    print("\n🗂️ [INFO] ▶️ 별도로 저장된 자막 파일:")
    for file in subtitle_files:
        if os.path.exists(file):
            print(f"  ✅ {os.path.abspath(file)}")
        else:
            print(f"  ⚠️ {file} 파일이 없습니다")


# 6️⃣ 다운로드된 불필요 파일 제거
def clean_temp_files(video_title):
    video_folder = os.path.join("downloads", video_title)
    temp_files = [
        os.path.join(video_folder, f"{video_title}.f140"),
        os.path.join(video_folder, f"{video_title}.f616")
    ]
    for temp_file in temp_files:
        if os.path.exists(temp_file):
            os.remove(temp_file)
            print(f"🧹 [INFO] ▶️ 불필요한 파일 삭제: {temp_file}")


# 7️⃣ 프로그램 실행
if __name__ == "__main__":
    os.makedirs("downloads", exist_ok=True)
    video_urls = [
        "https://www.youtube.com/watch?v=bl_35QzVsGU",
    ]

    for url in video_urls:
        # 1. 형식 및 자막 코드 확인
        has_subtitles = list_formats_and_subtitles(url)

        # 2. 동영상 다운로드 (자막 포함)
        if has_subtitles:
            video_title = download_video_with_subtitles(url)

            if video_title:
                # 3. 자막 파일 별도 보관
                backup_subtitles(video_title)

                # 4. 자막 파일 확인
                print_subtitle_location(video_title)

                # 5. 임시 파일 삭제
                clean_temp_files(video_title)
        else:
            print("🛑 [INFO] ▶️ 자막이 없는 영상이므로 다운로드를 건너뜁니다.")


[youtube] Extracting URL: https://www.youtube.com/watch?v=bl_35QzVsGU
[youtube] bl_35QzVsGU: Downloading webpage
[youtube] bl_35QzVsGU: Downloading ios player API JSON
[youtube] bl_35QzVsGU: Downloading tv player API JSON
[youtube] bl_35QzVsGU: Downloading m3u8 information

🖼️ [INFO] ▶️ 사용 가능한 영상 형식:
  🎞️ ID: sb3, 해상도: 48x27, 크기: Unknown bytes, 형식: mhtml
  🎞️ ID: sb2, 해상도: 80x45, 크기: Unknown bytes, 형식: mhtml
  🎞️ ID: sb1, 해상도: 160x90, 크기: Unknown bytes, 형식: mhtml
  🎞️ ID: sb0, 해상도: 320x180, 크기: Unknown bytes, 형식: mhtml
  🎞️ ID: 233, 해상도: audio only, 크기: Unknown bytes, 형식: mp4
  🎞️ ID: 234, 해상도: audio only, 크기: Unknown bytes, 형식: mp4
  🎞️ ID: 249-drc, 해상도: audio only, 크기: 18114003 bytes, 형식: webm
  🎞️ ID: 250-drc, 해상도: audio only, 크기: 22455731 bytes, 형식: webm
  🎞️ ID: 249, 해상도: audio only, 크기: 18090096 bytes, 형식: webm
  🎞️ ID: 250, 해상도: audio only, 크기: 22400331 bytes, 형식: webm
  🎞️ ID: 140-drc, 해상도: audio only, 크기: 46500166 bytes, 형식: m4a
  🎞️ ID: 251-drc, 해상도: audio only, 크기: 40900619 